In [ ]:
#Simple RGB full differential
from PIL import Image
import matplotlib.pyplot as plt
#Read image
img=Image.open('12WEBB-HP5-threeByTwoMediumAt2X.jpg') 
#img =img.transpose(Image.ROTATE_90)
#show image
plt.imshow(img)
plt.show()
img.size

In [ ]:
import numpy as np
w, h = img.size
#instrumentName = 'harp'  #actual sound must be chosen after importing MIDI file into a DAW
#note_str=['X','C2', 'C3', 'G3', 'C4', 'E4', 'G4', 'A#4', 'C5', 'D5', 'E5', 'F#5', 'G5', 'G#5'] #first 13 notes of the harmonic series
#instrumentName = 'drums'
#note_str=['X','C2', 'D2', 'E2', 'F#2', 'G#2', 'A#2', 'G2', 'A2', 'C3', 'D#3', 'E3', 'C#3', 'A3'] #drum 'notes'
# A, A#, B, C, C#, D, D#, E, F, F#, G, G#  #violon 'notes'
#note_str=['X','C1', 'C2', 'G2', 'C3', 'E3', 'G3', 'A3', 'B3', 'D4', 'E4', 'G4', 'A4', 'B4',
         #'D5', 'E5', 'G5', 'A5', 'B5','D6', 'E6', 'F#6', 'G6', 'A6',] #drum 'notes'
min_note=24
max_note=84
note_range =max_note-min_note   # you can try full note =>len(note_str)  (+some changes in the last cell)
small_img=img.resize((round(w/h*note_range),note_range),Image.BILINEAR)
plt.imshow(small_img)
np.array(small_img).shape

In [ ]:
img0 = np.asarray(np.array(small_img)[:,:,0])
#img0 = Image.fromarray(img0, 'L')
img1 = np.asarray(np.array(small_img)[:,:,1])
#img1 = Image.fromarray(img1, 'L')
img2 = np.asarray(np.array(small_img)[:,:,2])
#img2 = Image.fromarray(img2, 'L')

fig, axs = plt.subplots(1,3)
fig.suptitle('Horizontally stacked subplots')
axs[0].imshow(img0)
axs[0].set_title('R')
axs[1].imshow(img1)
axs[1].set_title('G')
axs[2].imshow(img2)
axs[2].set_title('B')


for ax in axs.flat:
    ax.set(xlabel='x-label', ylabel='y-label')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

In [21]:
pixels = np.array(small_img, dtype=int)
pixels.shape

(60, 90, 3)

In [22]:
differential = np.zeros(pixels.shape)
for k in range(0,pixels.shape[2]):
    for i in range(1,note_range):
        for j in range (1,round(w/h*note_range)):
            differential[i,j,k]=abs(pixels[i,j,k]-pixels[i,j-1,k])

In [ ]:
fig, axs = plt.subplots(1,3)
fig.suptitle('Horizontally stacked subplots')
axs[0].imshow(differential[:,:,0])
axs[0].set_title('R')
axs[1].imshow(differential[:,:,1])
axs[1].set_title('G')
axs[2].imshow(differential[:,:,2])
axs[2].set_title('B')


for ax in axs.flat:
    ax.set(xlabel='x-label', ylabel='y-label')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()
print(np.max(differential),np.min(differential),np.average(differential))

In [24]:
duration_scale= 64
min_velocity = 0
max_velocity = 127
velocity_range = max_velocity- min_velocity
x = differential
duration_img = small_img
pixels2 = np.round(np.array(duration_img, dtype=int) / duration_scale)
velocity = np.round(x / np.max(x) * velocity_range) + min_velocity

In [25]:
from midiutil import MIDIFile
from audiolazy import str2midi
bpm = 60
midifile = MIDIFile(adjust_origin=True)
midifile.addTempo(track=0, time=0, tempo=bpm)
for k in range(0,differential.shape[2]):
    for j in range(0,round(w/h*note_range)):
        for i in range(1,note_range): #pitch=str2midi(note_str[i])
            if velocity[i,j,k] >= 30:
                midifile.addNote(track=0, channel=0, pitch=i+min_note, time=j, duration=pixels2[i,j,k]+1, volume=int(velocity[i,j,k]))
    with open("srgbfd"+str(k)+".mid", "wb") as f:
        midifile.writeFile(f)

In [26]:
import cv2
import numpy as np
from PIL import Image
original_img= Image.open('12WEBB-HP5-threeByTwoMediumAt2X.jpg')
width = 920
height = 920
small_img = original_img.resize((width,height), Image.BILINEAR)
small_img.save('12WEBB-HP5-threeByTwoMediumAt2X.png')
fourcc = 0
video = cv2.VideoWriter('video.mp4', fourcc, 1, (width, height))
img = cv2.imread('12WEBB-HP5-threeByTwoMediumAt2X.png')
seconds = 96
for j in range(0,seconds):
    #img = cv2.imread('t1.jpg')
    video.write(img)
cv2.destroyAllWindows()
video.release()

In [27]:
# reading the input
from PIL import ImageDraw
cap = cv2.VideoCapture("video.mp4")
output = cv2.VideoWriter("moving.mp4",fourcc, 1, (width, height))
i = 0
while(True):
    ret, frame = cap.read()
    if(ret):
        i = i + 1
        # adding no fill rectangle on each frame
        #cv2.rectangle(frame, (int((i-1)/seconds*width),1), (int((i)/seconds*width),height-1),(255, 255, 255), 1)
        #adding transparent rectangle on each frame
        overlay = frame.copy()
        # Rectangle parameters
        x, y, w, h = int((i-1)/seconds*width),1, int(1.5/seconds*width), height-2
        # A filled rectangle
        cv2.rectangle(overlay, (x, y), (x+w, y+h), (255, 255, 255), -1)  
        alpha = 0.25  # Transparency factor.
        # Following line overlays transparent rectangle
        # over the image
        image_new = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
        # writing the new frame in output
        output.write(image_new)
        #cv2.imshow("output", frame)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break
    else:
        break
#print(i)
cv2.destroyAllWindows()
output.release()
cap.release()

In [28]:
import moviepy.editor as mpe
my_clip = mpe.VideoFileClip('moving.mp4')
audio_background = mpe.AudioFileClip('001_Sonification_of_12WEBBHP5threeByTwoMediumAt2X.mp3')
final_clip = my_clip.set_audio(audio_background)
final_clip.write_videofile("output.mp4",fps=30)

Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
